In [ ]:
# pip install pandas
# pip install geopandas

This block imports the required Python libraries:

pandas for handling tabular data (such as time series).
geopandas for working with geospatial data (GeoPackage format).
sqlite3 for connecting and interacting with an SQLite database.


In [1]:
# Import necessary libraries
import pandas as pd  # Pandas is used for handling tabular data efficiently
import geopandas as gpd  # Geopandas is used for handling geospatial data
import sqlite3  # SQLite3 is used to connect and interact with an SQLite database

This block loads three geospatial layers from a GeoPackage (.gpkg) file using geopandas:

POD_points: Contains the locations of Points of Diversion (POD), where water is withdrawn.
event: Represents gage stations or hydrological monitoring points.
ResOps_points: Includes information about reservoirs and their operations.
Each layer is stored as a GeoDataFrame, allowing spatial analysis and attribute queries.

In [2]:
# Read geospatial layers from a GeoPackage file
pod_layer = gpd.read_file('data/simple_enhanced_reference_14.gpkg', layer='POD_points')  
# POD (Point of Diversion) layer contains locations where water is diverted

gage_layer = gpd.read_file('data/simple_enhanced_reference_14.gpkg', layer='event')  
# Event layer contains gage locations or monitoring points

res_layer = gpd.read_file('data/simple_enhanced_reference_14.gpkg', layer='ResOps_points')  
# Reservoir operations layer contains details about reservoirs

add the start and end date for the simulation functionality 

In [2]:
start_date = '10/1/2003'  #specify start date for time series data
end_date = '10/1/2013'  #specify end date for time series data

This block establishes a connection to an SQLite database to query key identifiers. First, it retrieves the POI_TypeID for "POD" from the POI_Type table and the VariableID for "Demand" from the Variables table, both of which are crucial for querying related records.

Next, the script iterates through each diversion point in the pod_layer, extracting its Source_comid (hydrofabric segment ID) and using it to look up the corresponding POIID in the POI table. Once the POIID is found, it queries the POI_Values table to retrieve the historical demand time series, which is then converted into a pandas.DataFrame for easy processing. Additionally, it fetches water rights information, including the allocation date and the legally permitted water withdrawal rate (CFS) from the POD_WaterRights table. Next, script retrieves data such as the POI_NativeID (a unique identifier) and the POI_Flow_ComID (hydrofabric segment ID) from the POI table.

In each iteration, the script structures key variables for each point for integration into a water management model. These include the diversion point’s unique ID, hydrofabric segment, water rights information, and historical demand records. 

### <span style="color:red">There is a specified block in the code that can be used to implement MODEL parametrizing code.</span> 

Use the specified variables in each iteration to inject data to the model with model specific functions. 



In [5]:
# Establish connection to an SQLite database
conn = sqlite3.connect('data/optional_db.db')  
cursor = conn.cursor()  

# Query the POI_Type table to retrieve POI_TypeID for "POD" (Point of Diversion)
cursor.execute("SELECT POI_TypeID FROM POI_Type WHERE POI_TypeName = 'POD'")
poi_type_result = cursor.fetchone()  # Fetch the first matching result

# Check if a result was found; if not, print a message and exit
if poi_type_result:
    diversion_poi_type_id = poi_type_result[0]  # Extract POI_TypeID
else:
    print("No POI_TypeID found for Diversion. Check the POI_Type table.")
    conn.close()  # Close the database connection
    exit()  # Exit the script

# Query the Variables table to get the VariableID for "Demand"
cursor.execute("SELECT VariableID FROM Variables WHERE VariableName = 'Demand'")
variable_id_result = cursor.fetchone()  # Fetch the result

# Check if VariableID was found; if not, print a message and exit
if variable_id_result:
    variable_id = variable_id_result[0]  # Extract VariableID
else:
    print("No VariableID found for 'Diversion'. Check the Variables table.")
    conn.close()  # Close the database connection
    exit()  # Exit the script

# Iterate over each row (diversion point) in the POD layer
for index, row in pod_layer.iterrows():
    source_comid = row['Source_comid']  # Extract the Source_comid (hydrofabric segment ID)

    # Query the POI table to find the POIID associated with the current Source_comid
    cursor.execute("""
        SELECT POIID FROM POI 
        WHERE POI_Flow_ComID = ? AND POI_TypeID = ?
    """, (source_comid, diversion_poi_type_id))
    
    poiid_result = cursor.fetchone()  # Fetch the result

    if poiid_result:
        poiid = poiid_result[0]  # Extract POIID

        # Query the POI_Values table to get demand time series for this POI
        cursor.execute("""
            SELECT LocalDateTime, DataValue FROM POI_Values 
            WHERE POIID = ? AND VariableID = ? 
            ORDER BY LocalDateTime
        """, (poiid, variable_id))
        records = cursor.fetchall()  # Fetch time series records

        # Convert the retrieved records into a Pandas DataFrame
        demand_timeseries = pd.DataFrame(records, columns=['LocalDateTime', 'DiversionValue'])

        # Query the POD_WaterRights table to get allocation date and allocation value (CFS)
        cursor.execute("""
        SELECT WaterRightID, Allocation_Date, Allocation_CFS FROM POD_WaterRights
        WHERE POIID = ?
        """, (poiid,))
        waterrights_result = cursor.fetchall()  # Fetch all results
        
        # Query the POI table to retrieve the Native ID and the flow segment ID
        cursor.execute("""
        SELECT POI_NativeID, POI_Flow_ComID FROM POI
        WHERE POIID =?
        """, (poiid,))
        diversion_record = cursor.fetchall()  # Fetch the results

        diversion_ID = diversion_record[0][0]  # Extract POI Native ID
        segment_comid = diversion_record[0][1]  # Extract hydrofabric segment ID
        
        # The following prints each diversion point with all the water rights associated with it
        
        print()
        print (f"{diversion_ID}: ")
        for row in waterrights_result:
            WaterRightID = row[0]
            allocation_date = row[1]
            allocation_value = row[2]
            print(f"{WaterRightID}: {allocation_date}, {allocation_value}")
                     
        
        ##########################################################################
        #This block is for using the queried data in each iteration (diversion points) to implement data in MODEL
        # 1. `diversion_ID` - ID of the diversion point (POI)
        # 2. `segment_comid` - Hydrofabric segment ID where the diversion occurs
        # 3. 'WaterRightID' - The Native ID of the water right or allocation
        # 4. `allocation_date` - Date when water rights were allocated
        # 5. `allocation_value` - Water right allocation amount in CFS (use this or the historical demands)
        # 6. `demand_timeseries` - A Pandas DataFrame containing historical demand time series for this diversion point
        
        
        
        
        
        
        #########################################################################
        
conn.close()  # Close the database connection
 

3804617: 
30941.29454-0-C-3804617: 08/23/1930, 625.0
3800755: 
36763.0-0-C-3800755: 08/27/1950, 30.0
3800981: 
20041.19206-0-C-3800981: 08/02/1902, 58.0
51129.0-0-C-3800981: 12/26/1989, 1.0
3800755: 
36763.0-0-C-3800755: 08/27/1950, 30.0
3801594: 
39291.0-0-C-3801594: 07/29/1957, 0.0
3801790: 
14211.0-0-C-3801790: 11/27/1888, 13.0
12554.0-1-C-3801790: 05/15/1884, 1.04
25618.21642-0-C-3801790: 04/03/1909, 5.59
30941.23986-0-C-3801790: 09/03/1915, 4.31
51296.0-0-C-3801790: 06/11/1990, 1.2
56247.55497-1-C-3801790: 12/11/2001, 1.2
56536.0-1-C-3801790: 10/15/2004, 1.0
3804625: 
39291.0-0-C-3804625: 07/29/1957, 900.0
48577.39291-0-C-3804625: 07/29/1957, 100.0
3804613: 
28394.2611-0-C-3804613: 06/27/1921, 35.0
3800924: 
14045.0-0-C-3800924: 06/14/1888, 3.0
28989.28275-0-C-3800924: 06/01/1927, 0.5
28989.24273-0-C-3800924: 06/16/1916, 3.5
30941.28641-0-C-3800924: 06/01/1928, 18.0
3800924: 
14045.0-0-C-3800924: 06/14/1888, 3.0
28989.28275-0-C-3800924: 06/01/1927, 0.5
28989.24273-0-C-3800924: 06/

In [4]:
print(demand_timeseries.head())

  LocalDateTime  DiversionValue
0    1908-10-31             0.0
1    1908-11-30             0.0
2    1908-12-31             0.0
3    1909-01-31             0.0
4    1909-02-28             0.0
1233891
5104601
